In [289]:
%pylab inline
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.feature_selection import VarianceThreshold, chi2
from scipy.stats import chisquare

from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


# Installing Scikit-Feature 

http://featureselection.asu.edu/index.php

```
unzip scikit-feature-1.0.0.zip
cd scikit-feature-1.0.0
sudo python3.6 setup.py install
```

# Prerequisite

### Population and Sample

| 용어 | 한국말 | 의미 | 예제 |
|:----|:-----|:-----|:----|
| population | 모집단 | 전체 집단을 의미. | 30대 전체 한국 남성 |
| parameter | 모수 | 전체 population에 대한 summary number를 의미 | 30대 전체 한국 남성의 평균 몸무게 <br> mean $ \mu $, variance $  \sigma^2 $, std $ \sigma $ | 
| sample | 표본 | population으로부터 추출된 population을 대표하는 부분집합 | 30대 전체 한국 남성중 랜덤으로 선택된 100명 |
| statistic | 통계치 | sample에 대한 summary number | mean $ \bar{x} $, variance $ s^2 $, std $ s $|



### Confidency Interval

# Low Variance Filter

가장 쉽게 접근할수 있으며, 모델에 영향력을 갖고 있지 않는 데이터들을 빠르게 제거할 수 있는 방법입니다.<br>
VarianceThreshold는 지속적으로 zero-variance features 그리고 지속적으로 동일한 값을 내놓은 features들을 제거합니다.

### Example) Low variance for Bernoulli Distribution

boolean 데이터에 대해서 feature selection을 하려고 합니다. <br>
Success(True) 또는 Failure(False)같이 2개로만 나눠지는 경우 **Bernoulli Distribution**을 따릅니다. <br>

성공(True)값은 $ p $ 로 나타내며, 실패(False)는 $ 1 - p $ 로 정의를 합니다.<br>
Bernoulli Random Variable의 Variance구하는 공식은 다음과 같습니다.

$$ Var[X] = p(1-p) $$

아래의 예제에서는 80%이상이 0 또는 1이 지속되는 데이터를 삭제를 합니다.

In [170]:
def var(data):
    return round(np.var(data), 2)

# List of Variances
data = np.array([[0, 0, 1], 
                 [0, 1, 0], 
                 [1, 0, 0], 
                 [0, 1, 1], 
                 [0, 1, 0], 
                 [0, 1, 1]])

print('[0]:{0} \n[1]:{1} \n[2]:{2}'.format(var(data[:, 0]), var(data[:, 1]), var(data[:, 2])))

valsel = VarianceThreshold(threshold=0.8 * (1-0.8))  # 0.8 * (1-0.8) = 0.15999
valsel.fit_transform(data)

[0]:0.14 
[1]:0.22 
[2]:0.25


array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

### Example) Low variance for different variance range

Data1 ~ Data3 는 모두 normal distribution을 따르지만, Data1의 경우 0에 매우 가까우며, Data2의 경우 값 자체가 높습니다.<br>
Data1은 Variance Threshold 기준으로 variance값 자체가 작기때문에 feature selection에서 탈락하게 됩니다.

고루고루 값이 분포되어 있지만 variance값이 작아서 탈락이 된다면 Standardization 또는 MinMaxScaling을 해줍니다.<br>
[Normalization before low variance](http://www.kdnuggets.com/2015/05/7-methods-data-dimensionality-reduction.html) 를 참고 합니다.

> variance is range dependent; therefore normalization is required before applying this technique

**아래에서는 각 데이터의 variance값의 평균을 구한뒤 20% 보다 적은 variance값을 갖은 데이터를 제거합니다.**

In [234]:
x = np.random.normal(0, size=(50000, 4))
x[:, 0] /= 100
x[:, 1] += 10
x[:, 1] **= 2
x[:, 3] = 0
x[0:100, 3] = 1

print('[Before Scaling]')
print('Data1 Variance:', round(np.var(x[:, 0]), 5))
print('Data2 Variance:', round(np.var(x[:, 1]), 4))
print('Data3 Variance:', round(np.var(x[:, 2]), 4))
print('Data4 Variance:', round(np.var(x[:, 3]), 4))

x = MinMaxScaler().fit_transform(x)

print('\n[After Scaling]')
print('Data1 Variance:', round(np.var(x[:, 0]), 5))
print('Data2 Variance:', round(np.var(x[:, 1]), 5))
print('Data3 Variance:', round(np.var(x[:, 2]), 5))
print('Data4 Variance:', round(np.var(x[:, 3]), 5))

threshold = np.mean([np.var(x[:, i]) for i in range(4)]) *0.2

print('\nThresold:', threshold)
low_variance_feature_selection(x, threshold=threshold)

[Before Scaling]
Data1 Variance: 0.0001
Data2 Variance: 403.1223
Data3 Variance: 0.996
Data4 Variance: 0.002

[After Scaling]
Data1 Variance: 0.01315
Data2 Variance: 0.01556
Data3 Variance: 0.01388
Data4 Variance: 0.002

Thresold: 0.00222891870716


array([[ 0.43043046,  0.3634528 ,  0.40977524],
       [ 0.5702359 ,  0.4049609 ,  0.42229292],
       [ 0.57210535,  0.53095295,  0.566753  ],
       ..., 
       [ 0.51042491,  0.298668  ,  0.53970593],
       [ 0.56976632,  0.60402107,  0.43984602],
       [ 0.41341096,  0.67872648,  0.39401205]])

# Chi Sqaure Test

Chi sqaure test는 일반적으로 두가지 상황에서 사용이 될 수 있습니다. <br>
첫번째로는 Goodness-of-fit test(적합도 검정)로서 관측된 데이터가 예측한 분포를 따르는지 검정하는 방법입니다. <br>


$$ \chi^2_c = \sum \frac{ (O_i - E_i)^2 }{E_i} $$

* subscript $ c $ : **the degree of freedom** 
* $ O $ : observed value (관측값) 
* $ E $ : expected value 

Chi square는 observed value와 expected value사이가 얼마나 다른지를 타나내며, **categorical variables** 에서 사용됩니다.<br>
Chi square는 몇가지 variations들이 존재하며, 데이터 그리고 hypothesis에 따라서 적용이 달라집니다.

계산된 결과값이 낮으면 2개의 데이터셋 사이에는 높은 <span style="color:red">연관성</span>이 존재하며, <br>
observed value와 expected value가 서로 완전히 <span style="color:red">동일</span>하다면 (no difference) chi-square의 <span style="color:red">값은 0</span>이 됩니다. 

In [356]:
# Data
data = pd.DataFrame({'black': [9, 10, 12, 11, 8, 10], 
                     'red': [6, 5, 14, 15, 11, 9]}, 
                     index=range(1, 7))
display(data)
black = data.black.values
red = data.red.values

def my_chisquare(obs, exp):
    return np.sum((obs - exp)**2/exp)

# Calculate Chi-Square
chi_value, p_value = chisquare(black, black)
print('[Black]\tX-squared:', chi_value, '\tp-value:', round(p_value, 4))

chi_value, p_value = chisquare(red)
print('[Red]\tX-squared:', chi_value, '\tp-value:', round(p_value, 4))

print( my_chisquare(red, black))

,black,red
1,9,6
2,10,5
3,12,14
4,11,15
5,8,11
6,10,9


[Black]	X-squared: 0.0 	p-value: 1.0
[Red]	X-squared: 8.4 	p-value: 0.1355
6.51287878788


In [341]:
black = black
red = red

chi2(black.reshape(-1, 1), red)

(array([ 1.]), array([ 0.96256577]))

# T-Test